In [1]:
%reload_ext autoreload
%autoreload 2

import os
import urllib.request
import re


# from custom_tokenizer import *
import tokenizer  as tkn

In [3]:

# 处理文本，获得 tokenizer
if not os.path.exists("data/the-verdict.txt"):
    url  =("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

# get raw text 
with open("data/the-verdict.txt", "r", encoding='utf-8') as f:
    raw_text  = f.read()
print(f"len: {len(raw_text)}")

# split text
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f"len: {len(preprocessed)}")

# convert word into token
all_words = sorted(set(preprocessed)) 
vocab_size = len(all_words)
print(f"vocab_size: {vocab_size}")

# 得到字典
vocab = {word: i for i, word in enumerate(all_words)}
tokenizer = tkn.SimpleTokenizerV1(vocab)

# 测试 tokenizer
print("tokenizer: ", tokenizer)
# 测试编码
encoded = tokenizer.encode("I have mentioned that Mrs. Gisburn was rich")
print("encoded: ", encoded)
# 测试解码
decoded = tokenizer.decode(encoded)
print("decoded: ", decoded)

len: 20479
len: 4690
vocab_size: 1130


AttributeError: module 'tokenizer' has no attribute 'SimpleTokenizerV1'

In [10]:
# 处理之前没有出现的的 token
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
len(all_tokens)

vocab = {token: i for i, token in enumerate(all_tokens)}
for i, item in enumerate(list(vocab.items())[-5:]):
    print(f"{i}: {item}")

tokenizer = tkn.SimpleTokenizerV1(vocab)

# 测试新的 tokenizer
text = "hello world! This is a test."
encode = tokenizer.encode(text)
print(f"encode: {encode}")
decode = tokenizer.decode(encode)
print(f"decode: {decode}")

0: ('younger', 1127)
1: ('your', 1128)
2: ('yourself', 1129)
3: ('<|endoftext|>', 1130)
4: ('<|unk|>', 1131)
Preprocessed tokens: ['<|unk|>', '<|unk|>', '!', 'This', 'is', 'a', '<|unk|>', '.']
encode: [1131, 1131, 0, 97, 584, 115, 1131, 7]
decode: <|unk|> <|unk|>! This is a <|unk|>.


In [12]:
# byte pair encoding
import tiktoken
tiktoken.__version__

'0.9.0'

In [13]:
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
tokenizer.encode("hello world! This is a test.")
tokenizer.decode(tokenizer.encode("hello world! This is a test."))

'hello world! This is a test.'

## Data sampling with sliding window

In [6]:
import tiktoken
import torch
print(f"torch version: {torch.__version__}")

tiktoken.__version__

with open('data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
    
tokenizer = tiktoken.get_encoding("gpt2")   
encode_text = tokenizer.encode(raw_text)
print(len(encode_text))

torch version: 2.7.1+cu126
5145


In [7]:
encode_sample = encode_text[50:]
print(len(encode_sample))

context_size = 4
x = encode_sample[:context_size]
y = encode_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

5095
x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [8]:
for i in range(1, context_size + 1):
    context = encode_sample[:i]
    desired = encode_sample[i]
    
    print(f"{tokenizer.decode(context)} ---> {tokenizer.decode([desired])}")

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [9]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [10]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
with open("data/the-verdict.txt", "r", encoding='utf-8') as f:
    raw_text = f.read()
    
    
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
